# Datatón BC 2018 - Robocoach (Análisis de la variable ref1)

Se parte generando una base de datos en formato SQL con las tablas suministradas, esta base de datos está en la platafor AZURE de Microsoft y será cargada desde Google Colab por medio de un notebook de Python

La máquina virtual de Google Colab requiere la instalación de los siguientes paquetes para cargar la base de datos SQL 

In [1]:
!apt-get install libqt4-sql-odbc
!apt-get install unixodbc unixodbc-dev --install-suggests
!curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/18.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
!apt-get update
!ACCEPT_EULA=Y apt-get install msodbcsql17

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libqt4-sql-odbc is already the newest version (4:4.8.7+dfsg-7ubuntu1).
The following packages were automatically installed and are no longer required:
  autoconf autoconf-archive autoconf-doc automake autopoint gettext
  gettext-base gettext-doc gnu-standards libasprintf-dev libasprintf0v5
  libcroco3 libgettextpo-dev libgettextpo0 libsigsegv2 libtool-doc m4 m4-doc
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
unixodbc-dev is already the newest version (2.3.4-1.1ubuntu3).
unixodbc is already the newest version (2.3.4-1.1ubuntu3).
The following packages were automatically installed and are no longer required:
  autoconf autoconf-archive autoconf-doc automake autopoint gettext
  gettext-base gettext-doc gnu-standards libasprintf-dev libasprintf0v

In [2]:
 !pip install pyodbc

Carga de las librerías de Python utilizadas

In [0]:
import numpy as np
import pandas as pd
import pyodbc
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk import download
import string

Verificación de la instalación de pyodbc y del driver para gestionar SQL 

In [4]:
pyodbc.drivers()

['ODBC Driver 17 for SQL Server']

Definición de la conexión con el servidor de la base de datos de entrada

In [0]:
conn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};Server=tcp:robocoach.database.windows.net,1433;" \
                      "Database=datathon_database;Uid=datathon_admin@robocoach;Pwd=robocoach.2018;Encrypt=yes;"\
                      "TrustServerCertificate=yes;Connection Timeout=120;encoding='cp1252'")

Carga de la primera vista de los datos, esta se hace con el objetivo de definir las categorías que serán etiquetas de los datos. 
Se cargan las descripciones de todos los datos (transacciones) 

In [0]:
#sql = "SELECT DISTINCT ref1, COUNT(id_trn_ach) AS num_records FROM transacciones_personas_v GROUP BY ref1" # se demora 1061 s (18 min) segundos
sql = "SELECT ref1 FROM transacciones_personas_v"


data = pd.read_sql(sql,conn)

#print(data.to_string())

In [7]:
data.to_csv('ref1.csv')
data

,ref1
0,Pago de factura Gas Natural CupÃ³n No
1,Pago de factura Gas Natural CupÃ³n No
2,TARJETA DE CREDITO VISA Y AMEX
3,Pago de factura Gas Natural CupÃ³n No
4,pagos
5,Pago de factura Gas Natural CupÃ³n No
6,Pago facturas: CMR
7,Pago facturas: CMR
8,Pago de Saldo
9,CARTERA


Número total de datos

In [8]:
np.size(data)

11853782

Crear diccionario para reemplazar caractéres con tildes antes del vectorizer
(http://www.merinomartinez.com/2012/06/tabla-de-conversion-de-simbolos-utf-8-a-iso-8859-1-ansi/)

In [9]:
accents_dict = {
    #"Ã?": "A", 
    "Ã‰":"E", 
    "Ã": "I", 
    "Ã“": "O", 
    "Ãš": "U",
    "Ã¡": "a",
    "Ã³": "o", 
    "Ã©": "e",  
    "Ãº": "u",
    "Ã­": "i",
    "Ã‘": "Ñ",
    "Ã±": "ñ",
}

data.replace(to_replace=accents_dict, inplace=True, regex=True)
data

,ref1
0,Pago de factura Gas Natural Cupon No
1,Pago de factura Gas Natural Cupon No
2,TARJETA DE CREDITO VISA Y AMEX
3,Pago de factura Gas Natural Cupon No
4,pagos
5,Pago de factura Gas Natural Cupon No
6,Pago facturas: CMR
7,Pago facturas: CMR
8,Pago de Saldo
9,CARTERA


In [10]:
ref1_uniq = pd.DataFrame(data.ref1.unique(), columns=['ref1'], dtype='str')
ref1_uniq['ref1']

0                     Pago de factura Gas Natural  Cupon No
1                            TARJETA DE CREDITO VISA Y AMEX
2                                                     pagos
3                                        Pago facturas: CMR
4                                             Pago de Saldo
5                                                   CARTERA
6                                      Pago a traves de PSE
7                                                        CC
8                                PAGO ELECTRONICO EMCALIACH
9                   ContractId   Addreess  CALLE  #    PISO
10                                 Pago de factura Postpago
11                                                      CPV
12                                    Pago Factura Asociado
13        Presentacion y Pago del Impuesto Sobre Vehicul...
14                                          Pago de factura
15                      PAGO DE MONITOREO TELESENTINEL LTDA
16                             Pago de s

In [11]:
download('stopwords')
spanish_stop_words = stopwords.words('spanish')
spanish_stop_words.extend(['pago','psepayment','cc','no'])
spanish_stop_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosostros',
 'vosostras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestr

In [12]:
#data['ref1'] = data['ref1'].astype(str)
translator = str.maketrans('','', string.punctuation)
dictionary = ''.join([ref1 + " " for ref1 in ref1_uniq['ref1']])
dictionary.translate(string.punctuation)
dictionary

In [13]:
dict2 = dictionary.translate({ord(c): ' ' for c in '|_/:+%"#[](){}*@=?'})
dict2 = dict2.lower()
dict2

In [14]:
stemmer = SnowballStemmer('spanish')
stemmed_dictionary = ''.join([stemmer.stem(w) + " " for w in dict2.split() if w not in spanish_stop_words])
stemmed_dictionary

In [15]:
nonstemmed_dict = ''.join([w + " " for w in dict2.split() if w not in spanish_stop_words])
nonstemmed_dict

In [16]:
vectorizer = CountVectorizer(max_features=100)
vectorizer.fit_transform(nonstemmed_dict.split())
vectorizer.vocabulary_


{'avianca': 0,
 'booking': 1,
 'compra': 2,
 'date': 3,
 'eticket': 4,
 'fc': 5,
 'flight': 6,
 'paymentid': 7,
 'reserva': 8,
 'tiquete': 9}